## Computer Vision on Descartes Labs Platform:  Interactive Deployment with Dynamic Compute
__________________
This notebook will demonstrate a typical example of how to interact with the results of a deep learning model using Descartes Labs Platform APIs and define new AOIs to analyze on-the-fly.

The general steps covered in this notebook are:
* Retrieve a running [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function)
* Display results overlain on input imagery in a web map with [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html)
* Specify new areas to apply our model over with interactive [widgets](https://ipywidgets.readthedocs.io/en/stable/)

_**Note:**_ In order to run this example you must first complete the steps outlined in [03a Generate Training Data.ipynb](03a%20Generate%20Training%20Data.ipynb), [03b Training a Segmentation Model.ipynb](03b%20Training%20a%20Segmentation%20Model.ipynb), and [03c Deploying a Segmentation Model.ipynb](03c%20Deploying%20a%20Segmentation%20Model.ipynb).

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import properties as p
import descarteslabs.dynamic_compute as dc

from descarteslabs.compute import Function, FunctionStatus, Job
from descarteslabs.dynamic_compute import Mosaic
from descarteslabs.vector import Table

In [ ]:
import geopandas as gpd
from datetime import datetime
from ipyleaflet import DrawControl

### Retrieving an Active Compute Function 

If you lost your ID, you can retrieve it at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) or search the latest created Function with that name as below:

In [ ]:
func_search = (
    Function.search()
    .filter(p.owner == user_id)
    .filter(p.name.startswith("Deploy Wellpad Model"))
    .sort(-Function.creation_date)
    .limit(1)
).collect()

for func in func_search:
    print(func.id)
    print(func.creation_date)

In [ ]:
async_func = func_search[0]
async_func

### Setting Up Dynamic Compute

Here we will set  up the interactive map components to visualize our study area. 

First we set up a map frame alongside center coordinates and zoom level:

In [ ]:
m = dc.map

m.center = 33.5085, -101.5381
m.zoom = 14

Next create a and visualize a NAIP [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) for our time period:

In [ ]:
naip_mosaic = Mosaic.from_product_bands(
    "usda:naip:v1",
    "nir red green",
    start_datetime="2016-01-01",
    end_datetime="2017-01-01",
)
naip_mosaic.visualize("NAIP FCC", m)

And overlay our input training features as well:

In [ ]:
wellpad_table = Table.get("descarteslabs:wellpad-example-training-data")
wellpad_table.visualize("Wellpads", m)

## Interactive ipyleaflet Integration
Setting an empty list to keep track of all of our new [`Job`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job)s, and a simple handler function to add to our map. This function:
* Accepts a "draw" event, including the new geometry
* Splits the geometry into tiles and submits the tiles to our running function
* Extends the new jobs list with these newly submitted jobs

In [ ]:
new_jobs = []

In [ ]:
draw_control = DrawControl()


def handle_draw(target, action, geo_json):
    if action == "created":
        dltiles = dl.geo.DLTile.from_shape(
            geo_json, resolution=1.0, tilesize=512, pad=0
        )
        args = [(dltile.key, res_pid) for dltile in dltiles]
        jobs = async_func.map(args)
        new_jobs.extend(jobs)
        print(f"Submitted {len(jobs)} jobs")


draw_control.on_draw(handle_draw)
m.add_control(draw_control)

Lastly visualize our results:

In [ ]:
user_hash = dl.auth.Auth().namespace
org = dl.auth.Auth().payload['org']

res_pid = f"{org or  user_hash}:segmentation-outputs-{user_hash}"
res_pid

In [ ]:
res_mosaic = Mosaic.from_product_bands(res_pid, "class")
res_mosaic.visualize("Results", m)

And instantiate our map!

In [ ]:
m

Here we now have a draw control widget, which will submit newly drawn polygons to our compute function

In [ ]:
len(new_jobs)